In [1]:
import json
import os
from pathlib import Path
from google import genai
from google.genai import types
import time

In [2]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:

def load_story_metadata(json_file_path):
    """Load the story metadata from JSON file"""
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

def generate_images_for_story(json_file_path, output_dir="/home/mau/Documents/Projects/StoryForge/generated_images", overwrite=False):
    """
    Generate images from the JSON file for any story structure
    
    Args:
        json_file_path: Path to the JSON file
        output_dir: Directory to save images
        overwrite: If True, overwrites existing images. If False, skips existing files.
    """
    print(f"Loading story data from: {json_file_path}")
    
    # # Setup - you'll need to set your API key
    # GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
    # if not GEMINI_API_KEY:
    #     raise ValueError("Please set GOOGLE_API_KEY in your environment variables")
    
    client = genai.Client(api_key=GEMINI_API_KEY)
    
    # Load the story data
    story_data = load_story_metadata(json_file_path)
    
    # MADE GENERAL: Handle different JSON structures
    prompts_data = None
    story_id = None
    
    # Check if this is the new structure with 'stories' key
    if "stories" in story_data:
        # Get the first (and likely only) story
        stories = story_data["stories"]
        if not stories:
            print("No stories found in the JSON file")
            return
        
        # Get the first story ID
        story_id = list(stories.keys())[0]
        story_info = stories[story_id]
        
        if "image_prompts" in story_info:
            prompts_data = story_info["image_prompts"]
            print(f"Found story: {story_id}")
    
    # Check if this is the old flat structure
    elif "image_prompts" in story_data:
        prompts_data = story_data["image_prompts"]
        story_id = Path(json_file_path).stem  # Use filename as story ID
        print(f"Using flat structure for story: {story_id}")
    
    else:
        print("No image prompts found in the JSON file")
        return
    
    prompts = prompts_data.get("prompts", [])
    
    if not prompts:
        print("No prompts found in image_prompts")
        return
        
    print(f"Found {len(prompts)} image prompts to generate")
    
    # Create output directory
    story_dir = Path(output_dir) / story_id
    story_dir.mkdir(parents=True, exist_ok=True)
    
    # Track statistics
    generated_count = 0
    skipped_count = 0
    error_count = 0
    
    # Generate images for each prompt
    for i, prompt_data in enumerate(prompts):
        scene_number = prompt_data.get("scene_number", i+1)
        #title = prompt_data.get("title", f"Scene {scene_number}")
        prompt_text = prompt_data.get("description", "")
        story_part = prompt_data.get("story_part", 1)
        
        print(f"\nGenerating Scene {scene_number}")
        print(f"Part {story_part} | Prompt preview: {prompt_text[:100]}...")
        
        try:
            # Generate images using Imagen 3
            response = client.models.generate_images(
                model='imagen-3.0-generate-002',
                prompt=prompt_text,
                config=types.GenerateImagesConfig(
                    number_of_images=1,
                    aspectRatio="9:16"
                )
            )
            
            # Save the generated images
            for img_idx, generated_image in enumerate(response.generated_images):
                # Create filename
                #safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
                filename = f"scene_{scene_number:02d}_v{img_idx+1}.png"
                filepath = story_dir / filename
                
                # CHECK IF FILE EXISTS AND HANDLE OVERWRITE
                if filepath.exists() and not overwrite:
                    print(f"  ⏭ Skipped (already exists): {filepath}")
                    skipped_count += 1
                    continue
                
                # Save the image
                generated_image.image.save(str(filepath))
                
                if filepath.exists() and not overwrite:
                    print(f"  ✓ Generated: {filepath}")
                else:
                    print(f"  ✓ Overwritten: {filepath}")
                generated_count += 1
            
            # Add a small delay to avoid rate limiting
            time.sleep(2)
            
        except Exception as e:
            print(f"✗ Error generating image for Scene {scene_number}: {str(e)}")
            error_count += 1
            continue
    
    # Print summary
    print(f"\n🎉 Image generation complete!")
    print(f"📁 Images saved to: {story_dir}")
    print(f"📊 Summary:")
    print(f"   - Generated: {generated_count} images")
    print(f"   - Skipped: {skipped_count} images")
    print(f"   - Errors: {error_count} scenes")

def list_stories_in_json(json_file_path):
    """
    List all stories available in the JSON file
    """
    story_data = load_story_metadata(json_file_path)
    
    if "stories" in story_data:
        stories = story_data["stories"]
        print(f"Found {len(stories)} story(ies):")
        for story_id, story_info in stories.items():
            prompt_count = story_info.get("image_prompts", {}).get("count", 0)
            duration = story_info.get("duration", 0)
            print(f"  - {story_id}: {prompt_count} prompts, {duration:.1f}s duration")
    else:
        print("Single story format detected")
        prompt_count = story_data.get("image_prompts", {}).get("count", 0)
        print(f"  - {prompt_count} prompts available")

# Example usage
if __name__ == "__main__":
    # List what stories are available
    list_stories_in_json("/home/mau/Documents/Projects/StoryForge/generated_images/story_metrics_story_18_20250606_203132/story_metrics_story_20250606_203132.json")
    
    # Generate images (won't overwrite by default)
    generate_images_for_story("/home/mau/Documents/Projects/StoryForge/generated_images/story_metrics_story_18_20250606_203132/story_metrics_story_20250606_203132.json", overwrite=True)

Single story format detected
  - 10 prompts available
Loading story data from: /home/mau/Documents/Projects/StoryForge/generated_images/story_metrics_story_18_20250606_203132/story_metrics_story_20250606_203132.json
Using flat structure for story: story_metrics_story_20250606_203132
Found 10 image prompts to generate

Generating Scene 1
Part 1 | Prompt preview: Within the dim, underwater depths of a corroded submarine wreck, a clever octopus, eyes gleaming wit...


KeyboardInterrupt: 